In [1]:
import sys
import weakref

## Recall how reference count work

In [3]:
class MyObject:
    pass

In [4]:
o = MyObject()
print(f"reference count {o} : {sys.getrefcount(o)-1}")

reference count <__main__.MyObject object at 0x000001CEE4E0FB30> : 1


In [5]:
o_bis = o
print(f"reference count {o} : {sys.getrefcount(o)-1}")

reference count <__main__.MyObject object at 0x000001CEE4E0FB30> : 2


In [6]:
 o_ref = weakref.ref(o)
o_ref

<weakref at 0x000001CEE607DAD0; to 'MyObject' at 0x000001CEE4E0FB30>

In [7]:
# when creating a weak reference to an object, it doesn't incrase the reference count
print(f"reference count {o} : {sys.getrefcount(o)-1}")

reference count <__main__.MyObject object at 0x000001CEE4E0FB30> : 2


In [8]:
# however when calling the weak reference, it creates a "hard reference" and increase the reference count
o_ter = o_ref()
print(f"reference count {o} : {sys.getrefcount(o)-1}")

reference count <__main__.MyObject object at 0x000001CEE4E0FB30> : 3


## Using a weakref cache

Once again the smart trick is to store a weakref of the object in the cache once created. Then, when called upon, lookup in the cache and return a hard
ref of this object from the weak reference by calling weak_ref().

In [10]:
class Date:
    _cache = { }
    
    @staticmethod
    def __new__(cls, year, month, day):
        
        selfref = Date._cache.get((year,month,day))
        if not selfref:
            self = super().__new__(cls)
            self.year = year
            self.month = month
            self.day = day
            Date._cache[year,month,day] = weakref.ref(self) # return a hard ref to the object
        else:
            self = selfref() # return a hard ref to the object
            
        return self
    
    def __init__(self, year, month, day):
        pass
        
    def __del__(self):
        print(f"{self} delete")
        del Date._cache[self.year,self.month,self.day] # clear cache

In [11]:
a = Date(2022, 12,3)
print(f"reference count {a} : {sys.getrefcount(a)-1}")

reference count <__main__.Date object at 0x000001CEE4A1A0F0> : 1


In [12]:
b = a # look up in dict and create a hard ref by calling weakref.ref()
print(f"reference count {a} : {sys.getrefcount(a)-1}")

reference count <__main__.Date object at 0x000001CEE4A1A0F0> : 2


In [13]:
Date._cache

{(2022,
  12,
  3): <weakref at 0x000001CEE607EE80; to 'Date' at 0x000001CEE4A1A0F0>}

In [14]:
a is b

True

In [15]:
del b

In [16]:
print(f"reference count {a} : {sys.getrefcount(a)-1}")

reference count <__main__.Date object at 0x000001CEE4A1A0F0> : 1


In [17]:
del a

<__main__.Date object at 0x000001CEE4A1A0F0> delete


In [18]:
Date._cache

{}